In [1]:
import numpy as np
import pandas as pd

import seaborn as sns

In [2]:
enhancer_gene_pairs = pd.read_csv('/iblm/netapp/data1/external/Gasperini2019/suppl/GSE120861_gene_gRNAgroup_pair_table.at_scale.txt', sep = '\t')

/tmp/ipykernel_823/1374250479.py:1: DtypeWarning: Columns (1,2) have mixed types. Specify dtype option on import or set low_memory=False.
  enhancer_gene_pairs = pd.read_csv('/iblm/netapp/data1/external/Gasperini2019/suppl/GSE120861_gene_gRNAgroup_pair_table.at_scale.txt', sep = '\t')


In [3]:
enhancer_gene_pairs.head()

,gRNAgroup.chr,gRNAgroup.start,gRNAgroup.stop,gRNAgroup,general_group,chr.targetgene,start.targetgene,stop.targetgene,ENSG.targetgene,targetgene_short_name,strand.targetgene,pairs
0,NTC,NTC,NTC,bassik_mch,NTC,chr10,28034777,28034778,ENSG00000150051,MKX,-,MKX:bassik_mch
1,NTC,NTC,NTC,bassik_mch,NTC,chr10,28287976,28287977,ENSG00000169126,ARMC4,-,ARMC4:bassik_mch
2,NTC,NTC,NTC,bassik_mch,NTC,chr10,28571017,28571018,ENSG00000150054,MPP7,-,MPP7:bassik_mch
3,NTC,NTC,NTC,bassik_mch,NTC,chr10,28821422,28821423,ENSG00000095787,WAC,+,WAC:bassik_mch
4,NTC,NTC,NTC,bassik_mch,NTC,chr10,28966271,28966272,ENSG00000095739,BAMBI,+,BAMBI:bassik_mch


In [4]:
enhancers = enhancer_gene_pairs[['gRNAgroup.chr', 'gRNAgroup.start', 'gRNAgroup.stop', 'gRNAgroup']]
enhancers = enhancers.drop_duplicates().reset_index(drop=True)
enhancers.head()

,gRNAgroup.chr,gRNAgroup.start,gRNAgroup.stop,gRNAgroup
0,NTC,NTC,NTC,bassik_mch
1,NTC,NTC,NTC,random_1
2,NTC,NTC,NTC,random_10
3,NTC,NTC,NTC,random_11
4,NTC,NTC,NTC,random_12


In [5]:
enhancers['gRNAgroup.start'] = enhancers['gRNAgroup.start'].replace({'NTC': np.nan}).astype(np.float64)
enhancers['gRNAgroup.stop'] = enhancers['gRNAgroup.stop'].replace({'NTC': np.nan}).astype(np.float64)


In [6]:
enhancers['position'] = (enhancers['gRNAgroup.start'] + enhancers['gRNAgroup.stop']) / 2
enhancers.head()

,gRNAgroup.chr,gRNAgroup.start,gRNAgroup.stop,gRNAgroup,position
0,NTC,NaN,NaN,bassik_mch,NaN
1,NTC,NaN,NaN,random_1,NaN
2,NTC,NaN,NaN,random_10,NaN
3,NTC,NaN,NaN,random_11,NaN
4,NTC,NaN,NaN,random_12,NaN


In [7]:
enhancers = enhancers[['gRNAgroup', 'position']]
enhancers.columns = ['enhancer', 'position']
enhancers.head()

,enhancer,position
0,bassik_mch,NaN
1,random_1,NaN
2,random_10,NaN
3,random_11,NaN
4,random_12,NaN


In [8]:
enhancers['enhancer'] = enhancers['enhancer'].apply(lambda x: x.split('_')[0])
enhancers.head()

,enhancer,position
0,bassik,NaN
1,random,NaN
2,random,NaN
3,random,NaN
4,random,NaN


In [9]:
enhancers[enhancers['enhancer'] == 'chr11.1734']

,enhancer,position
482,chr11.1734,33963390.5
483,chr11.1734,33963390.5


In [10]:
enhancers = enhancers.drop_duplicates()

In [12]:
enhancer_enhancer_results = pd.read_csv('/iblm/netapp/data1/external/Gasperini2019/processed/23_01_12_enhancer_enhancer_pairs_suppl_table_2_pseudocount_model_enhancer_effects.csv')
enhancer_enhancer_results.head()

,enhancer.1.list,enhancer.2.list,gene.list,enhancer.1.coeff.list,enhancer.1.pvalue.list,enhancer.2.coeff.list,enhancer.2.pvalue.list,interaction.coeff.list,interaction.pvalue.list
0,chr7.4040,chr7.4045,ENSG00000005249,-0.494266,2.900198e-26,-0.545437,2.234040e-19,-0.233471,0.850829
1,chr7.4040,chr7.4046,ENSG00000005249,-0.493618,3.504769e-26,-0.332388,1.180364e-15,-0.799996,0.480200
2,chr7.4040,chr7.4041,ENSG00000005249,-0.494144,4.073117e-26,-0.359455,2.127851e-10,0.179882,0.814348
3,chr7.4040,chr7.4042,ENSG00000005249,-0.496815,2.557499e-26,-0.204326,4.428635e-06,0.436987,0.488639
4,chr7.4040,chr7.4048,ENSG00000005249,-0.492068,4.706762e-26,-0.172718,1.340911e-02,-0.981096,0.480882


In [13]:
enhancer_enhancer_results = enhancer_enhancer_results.merge(enhancers, how = 'left', left_on = 'enhancer.1.list', right_on = 'enhancer').rename(columns = {'enhancer': 'enhancer.1.group', 'position': 'enhancer.1.position'})
enhancer_enhancer_results = enhancer_enhancer_results.merge(enhancers, how = 'left', left_on = 'enhancer.2.list', right_on = 'enhancer').rename(columns = {'enhancer': 'enhancer.2.group', 'position': 'enhancer.2.position'})
enhancer_enhancer_results.head()

,enhancer.1.list,enhancer.2.list,gene.list,enhancer.1.coeff.list,enhancer.1.pvalue.list,enhancer.2.coeff.list,enhancer.2.pvalue.list,interaction.coeff.list,interaction.pvalue.list,enhancer.1.group,enhancer.1.position,enhancer.2.group,enhancer.2.position
0,chr7.4040,chr7.4045,ENSG00000005249,-0.494266,2.900198e-26,-0.545437,2.234040e-19,-0.233471,0.850829,chr7.4040,106619094.5,chr7.4045,106647836.0
1,chr7.4040,chr7.4046,ENSG00000005249,-0.493618,3.504769e-26,-0.332388,1.180364e-15,-0.799996,0.480200,chr7.4040,106619094.5,chr7.4046,106650738.0
2,chr7.4040,chr7.4041,ENSG00000005249,-0.494144,4.073117e-26,-0.359455,2.127851e-10,0.179882,0.814348,chr7.4040,106619094.5,chr7.4041,106619758.5
3,chr7.4040,chr7.4042,ENSG00000005249,-0.496815,2.557499e-26,-0.204326,4.428635e-06,0.436987,0.488639,chr7.4040,106619094.5,chr7.4042,106620326.5
4,chr7.4040,chr7.4048,ENSG00000005249,-0.492068,4.706762e-26,-0.172718,1.340911e-02,-0.981096,0.480882,chr7.4040,106619094.5,chr7.4048,106666174.0


In [14]:
enhancer_enhancer_results['distance'] = np.abs(enhancer_enhancer_results['enhancer.1.position'] - enhancer_enhancer_results['enhancer.2.position'])
enhancer_enhancer_results.head()

,enhancer.1.list,enhancer.2.list,gene.list,enhancer.1.coeff.list,enhancer.1.pvalue.list,enhancer.2.coeff.list,enhancer.2.pvalue.list,interaction.coeff.list,interaction.pvalue.list,enhancer.1.group,enhancer.1.position,enhancer.2.group,enhancer.2.position,distance
0,chr7.4040,chr7.4045,ENSG00000005249,-0.494266,2.900198e-26,-0.545437,2.234040e-19,-0.233471,0.850829,chr7.4040,106619094.5,chr7.4045,106647836.0,28741.5
1,chr7.4040,chr7.4046,ENSG00000005249,-0.493618,3.504769e-26,-0.332388,1.180364e-15,-0.799996,0.480200,chr7.4040,106619094.5,chr7.4046,106650738.0,31643.5
2,chr7.4040,chr7.4041,ENSG00000005249,-0.494144,4.073117e-26,-0.359455,2.127851e-10,0.179882,0.814348,chr7.4040,106619094.5,chr7.4041,106619758.5,664.0
3,chr7.4040,chr7.4042,ENSG00000005249,-0.496815,2.557499e-26,-0.204326,4.428635e-06,0.436987,0.488639,chr7.4040,106619094.5,chr7.4042,106620326.5,1232.0
4,chr7.4040,chr7.4048,ENSG00000005249,-0.492068,4.706762e-26,-0.172718,1.340911e-02,-0.981096,0.480882,chr7.4040,106619094.5,chr7.4048,106666174.0,47079.5


In [15]:
enhancer_enhancer_results[enhancer_enhancer_results['enhancer.1.list'] == 'chr11.1735']

,enhancer.1.list,enhancer.2.list,gene.list,enhancer.1.coeff.list,enhancer.1.pvalue.list,enhancer.2.coeff.list,enhancer.2.pvalue.list,interaction.coeff.list,interaction.pvalue.list,enhancer.1.group,enhancer.1.position,enhancer.2.group,enhancer.2.position,distance
127,chr11.1735,chr11.1734,ENSG00000121691,-0.655152,3.682906e-32,-0.147817,5.209850e-04,-0.112404,0.874983,chr11.1735,33966493.5,chr11.1734,33963390.5,3103.0
141,chr11.1735,chr11.1734,ENSG00000135363,-1.820761,1.333543e-222,-1.002767,1.398763e-118,2.117515,0.000765,chr11.1735,33966493.5,chr11.1734,33963390.5,3103.0
142,chr11.1735,chr11.1733,ENSG00000135363,-1.800177,6.430887e-220,-0.373082,1.246321e-14,-0.404461,0.684963,chr11.1735,33966493.5,chr11.1733,33958891.5,7602.0
143,chr11.1735,chr11.1732,ENSG00000135363,-1.803272,2.758996e-220,-0.186463,2.152756e-04,0.253622,0.785528,chr11.1735,33966493.5,chr11.1732,33954506.5,11987.0
144,chr11.1735,chr11.1764,ENSG00000135363,-1.802154,1.734683e-220,-0.162506,1.540369e-02,0.117392,0.918621,chr11.1735,33966493.5,chr11.1764,34393196.5,426703.0
145,chr11.1735,chr11.1780,ENSG00000135363,-1.803527,1.140683e-221,-0.220565,3.217863e-04,0.704303,0.744264,chr11.1735,33966493.5,chr11.1780,34640740.5,674247.0


In [16]:
enhancer_enhancer_results.sort_values(by = 'interaction.pvalue.list').head(10)

,enhancer.1.list,enhancer.2.list,gene.list,enhancer.1.coeff.list,enhancer.1.pvalue.list,enhancer.2.coeff.list,enhancer.2.pvalue.list,interaction.coeff.list,interaction.pvalue.list,enhancer.1.group,enhancer.1.position,enhancer.2.group,enhancer.2.position,distance
141,chr11.1735,chr11.1734,ENSG00000135363,-1.820761,1.333543e-222,-1.002767,1.398763e-118,2.117515,0.000765,chr11.1735,33966493.5,chr11.1734,33963390.5,3103.0
310,chrX.233,chrX.239,ENSG00000205542,-1.298632,4.380139e-223,-0.386695,2.078205e-11,-4.158882,0.001105,chrX.233,12975150.0,chrX.239,12996541.0,21391.0
324,chrX.231,chrX.230,ENSG00000205542,-0.268157,1.075408e-10,-0.176616,9.977053e-04,-3.070143,0.003074,chrX.231,12969185.5,chrX.230,12968495.0,690.5
239,chr6.4960,chr6.4971,ENSG00000164442,-0.406697,3.930360e-06,-0.367024,3.713939e-06,4.534203,0.006567,chr6.4960,139770351.0,chr6.4971,139946662.5,176311.5
120,chr2.183,chr2.184,ENSG00000115738,-0.637552,1.321578e-28,-0.536502,2.113209e-09,2.366853,0.007796,chr2.183,8666020.5,chr2.184,8675282.0,9261.5
328,chrX.230,chrX.240,ENSG00000205542,-0.178957,8.411369e-04,-0.126285,3.255201e-02,-3.908348,0.012751,chrX.230,12968495.0,chrX.240,13003150.5,34655.5
64,chrX.504,chrX.505,ENSG00000099399,-0.859755,5.533806e-04,-0.628958,3.529793e-04,6.563903,0.022037,chrX.504,30301913.0,chrX.505,30302447.0,534.0
132,chr22.1657,chr22.1656,ENSG00000128309,-0.174576,2.990948e-10,-0.162303,6.543356e-06,-1.223476,0.027802,chr22.1657,37378405.5,chr22.1656,37376926.5,1479.0
248,chr16.1866,chr16.1865,ENSG00000166501,-1.130722,2.490490e-198,-0.185285,5.204579e-07,0.964922,0.031429,chr16.1866,23833442.5,chr16.1865,23815022.0,18420.5
106,chr11.3243,chr11.3251,ENSG00000110046,-0.140396,2.626002e-01,-0.319538,1.132761e-01,3.316676,0.033107,chr11.3243,65239699.5,chr11.3251,65261483.0,21783.5


In [17]:
enhancer_enhancer_results.to_csv('/iblm/netapp/data1/external/Gasperini2019/processed/23_04_25_enhancer_distance_330_pairs.csv', index=False)